# Traveling Waves Search

2025-10-13

Eben Quenneville

Searching for traveling waves in the low-d model by projecting known solutions downward. 



In [1]:
# Polynomials, StaticArrays, OffsetArrays, LinearAlgebra, SparseArrays, StringBuilders, Revise
import Pkg
using LinearAlgebra
using SparseArrays
using NLsolve
using Dates
using Revise
using Polynomials
using Profile
using DelimitedFiles
import Base.@kwdef
myreaddlm(filename) = readdlm(filename, comments=true, comment_char='%');

In [2]:
includet("src/BasisFunctions.jl")
includet("src/BifurcationGenerator.jl")
includet("src/Hookstep.jl")
includet("src/tw_newton.jl")

using .BasisFunctions
using .BifurcationGenerator
using .Hookstep
using .TW_Newton

In [3]:
@kwdef struct TWConfig
    name::Symbol
    Re_values::Vector{Int}
    cx0::Float64
    cz0::Float64
    symmetry_group::SymmetryGroup
end

TWConfig

In [4]:
const TW_CONFIGS = Dict(
    :TW1 => TWConfig(
        name = :TW1,
        Re_values = [200, 300],
        cx0 = 0.0,
        cz0 = 0.009,
        symmetry_group = SymmetryGroup(:TW1)
    ),
    :TW2 => TWConfig(
        name = :TW2,
        Re_values = [250, 300],
        cx0 = 0.3959,
        cz0 = 0.0,
        symmetry_group = SymmetryGroup(:TW2)
    ),
    :TW3 => TWConfig(
        name = :TW3,
        Re_values = [250, 300],
        cx0 = 0.4646,
        cz0 = 0.0,
        symmetry_group = SymmetryGroup(:TW2)
    )
)

Dict{Symbol, TWConfig} with 3 entries:
  :TW2 => TWConfig(:TW2, [250, 300], 0.3959, 0.0, SymmetryGroup(:TW2))
  :TW1 => TWConfig(:TW1, [200, 300], 0.0, 0.009, SymmetryGroup(:TW1))
  :TW3 => TWConfig(:TW3, [250, 300], 0.4646, 0.0, SymmetryGroup(:TW2))

In [5]:
# Global Parameters
α, γ = 1, 2
ftol = 1e-8
max_iterations = 50

50

In [6]:
function run_parameter_search(mode::Symbol, JKL::NTuple{3, Int})
    config = TW_CONFIGS[mode]

    output = []
    
    for Re in config.Re_values
        println("Running $(config.name) at Re=$Re, JKL=$JKL")
        println("cx0=$(config.cx0), cz0=$(config.cz0), symmetry=$(config.symmetry_group)")
        Ψ, ijkl = generate_psi_ijkl(JKL; α=α, γ=γ, symmetry=config.symmetry_group)
        A_of_Re, Cx, Cz, N = tw_matrices(Ψ)
        A = A_of_Re(Re)
        Nmodes = length(Ψ)

        # Load initial guess
        x0 = myreaddlm("data/continued_tws/starts/x$(config.name)-2pi1piRe$(Re)-$(JKL[1])-$(JKL[2])-$(JKL[3]).asc")
        cx0 = config.cx0
        cz0 = config.cz0
        z0 = [x0; cx0; cz0]

        println("Starting Residual: ", norm(residual(x0, cx0, cz0, A, Cx, Cz, N)))

        println("Solving with hookstep...")
        hookstep_sol, hookstep_cx, hookstep_cz, hookstep_iterates = newton_hookstep_bordered!(x0, cx0, cz0; A=A, Cx=Cx, Cz=Cz, N=N, verbose=false)
        println("Search complete.")
    
        println("Norm of solution: ", norm(hookstep_sol))
        println("Wave speed in x: ", hookstep_cx)
        println("Wave speed in z: ", hookstep_cz)

        println("\n\nNow verifying solution by a timestepped approximation:")
        
        verified = verify(hookstep_sol, hookstep_cx, hookstep_cz, (A, Cx, Cz, N))

        push!(output, (hookstep_sol, hookstep_cx, hookstep_cz, hookstep_iterates))
        println("\nRun complete.\n\n")
    end
    return output
end

run_parameter_search (generic function with 1 method)

# Hookstep Solving

In [7]:
JKL = (1, 2, 2)
TW = :TW1
solutions = run_parameter_search(TW, JKL)

Running TW1 at Re=200, JKL=(1, 2, 2)
cx0=0.0, cz0=0.009, symmetry=SymmetryGroup(:TW1)
Starting Residual: 0.015475524055859046
Solving with hookstep...
Search complete.
Norm of solution: 0.38669213836496336
Wave speed in x: 0.0
Wave speed in z: -3.693917763092323e-9


Now verifying solution by a timestepped approximation:
Starting time evolution...
Evolution finished at T = 5.0
Calculating analytically shifted state...

--- Verification Results ---
L2 Error between simulated and analytically shifted wave: 1.3220029266320572e-9
Result: The solution behaves like a true traveling wave.

Run complete.


Running TW1 at Re=300, JKL=(1, 2, 2)
cx0=0.0, cz0=0.009, symmetry=SymmetryGroup(:TW1)
Starting Residual: 0.010829371591070556
Solving with hookstep...
Search complete.
Norm of solution: 0.3835110139471293
Wave speed in x: 0.0
Wave speed in z: -6.777753228249082e-10


Now verifying solution by a timestepped approximation:
Starting time evolution...
Evolution finished at T = 5.0
Calculating an

2-element Vector{Any}:
 ([0.11684224250940252, 3.797615293780035e-9, -8.546098107018524e-10, -0.34879289189751583, -0.0280023812731161, -0.009612963707434648, 0.0614400170352947, 0.004932630938007628, -0.0016933282175598024, -0.04644530942279447  …  -0.004025796301040019, 0.002382221188323836, 0.02285429854349074, -0.013523787348818003, -0.006153719702995323, -0.012633661532235886, 0.003138570803113872, 0.016925242238042038, 0.034747718592264824, -0.008632346932315642], 0.0, -3.693917763092323e-9, [0.1619020883776885 0.024721049551058586 … 0.0 0.009; 0.1360547962150119 0.019702614968415702 … 0.0 -0.011926331590752527; … ; 0.11681359444275968 4.740184433116605e-6 … 0.0 -5.576215197360124e-6; 0.11684224250940252 3.797615293780035e-9 … 0.0 -3.693917763092323e-9])
 ([0.1040652225801688, 9.32068171939251e-10, 1.2031046319198985e-10, -0.34800772022021303, -0.05344167522579536, -0.006451670709934224, 0.06800121414826094, 0.010442578335854316, -0.0012606659347928523, -0.05685043439334534  …  -

In [8]:
println("Saving results...")
for (i, Re) in enumerate(TW_CONFIGS[TW].Re_values)
    sol, cx, cz, iterates = solutions[i]
    J, K, L = JKL
    println("Saving to $(TW)-2pi1piRe$(Re)-$J-$K-$L-sol.asc")
    save(sol, "$(TW)-2pi1piRe$(Re)-$J-$K-$L-sol.asc")
    save_sigma(cx, cz, 10, "$(TW)-2pi1piRe$(Re)-$J-$K-$L-sol-sigma.asc")
end

Saving results...
Saving to TW1-2pi1piRe200-1-2-2-sol.asc
Saving to TW1-2pi1piRe300-1-2-2-sol.asc


In [ ]:
for TW in (:TW2, )
    for JKL in ((4, 4, 8), ) # (1, 2, 2), (2, 2, 4), (3, 3, 6), 
        solutions = run_parameter_search(TW, JKL)
        println("Saving results...")
        for (i, Re) in enumerate(TW_CONFIGS[TW].Re_values)
            sol, cx, cz, iterates = solutions[i]
            J, K, L = JKL
            println("Saving to $(TW)-2pi1piRe$(Re)-$J-$K-$L-sol.asc")
            save(sol, "$(TW)-2pi1piRe$(Re)-$J-$K-$L-sol.asc")
            save_sigma(cx, cz, 10, "$(TW)-2pi1piRe$(Re)-$J-$K-$L-sol-sigma.asc")
        end
    end
end

# Traveling Wave Derivation

The Navier-Stokes equations are

$$
\begin{align}
  \dfrac{\partial u_{tot}}{\partial t} = -\nabla p + \dfrac{1}{Re} \nabla^2 u_{tot} - u_{tot} \cdot \nabla u_{tot}
\end{align}
$$

Assume a doubly-periodic domain $\Omega = [0,L_x) \times [-1, 1] \times [0,L_z)$ with periodic boundary conditions in $x,z$ and no-slip conditions $u_{tot} = \pm1$ at the walls $y=\pm1$. 

Let $u_{tot} = y e_x + u$. This breaks the total velocity into sum of the laminar solution
$y e_x$ (the base flow) and a fluctuation $u$. Then $\nabla^2 u_{tot} = \nabla^2 u$,

$$
\begin{align} 
u_{tot} \cdot \nabla u_{tot} 
 &=  (y e_x + u) \cdot \nabla (y e_x + u) \\
 &=  y \dfrac{\partial u}{\partial x} + v e_x + u \cdot \nabla u,
\end{align}
$$
and
$$
\begin{align}
  \dfrac{\partial u}{\partial t} = -\nabla p - y \dfrac{\partial u}{\partial x} - v e_x + \dfrac{1}{Re} \nabla^2 u - u \cdot \nabla u
\end{align}
$$

Replacing the time derivative $\dfrac{\partial u}{\partial t}$ with the spatial derivative term $c_x \dfrac{\partial u}{\partial x} + c_z \dfrac{\partial u}{\partial z}$ transforms the governing equation for the fluctuation $u$ into:

$$c_x \dfrac{\partial u}{\partial x} + c_z \dfrac{\partial u}{\partial z} = -\nabla p - y \dfrac{\partial u}{\partial x} - v e_x + \dfrac{1}{Re} \nabla^2 u - u \cdot \nabla u$$

Since the Navier–Stokes equations for plane Couette flow are unchanged by shifts in the $x$ and $z$ directions, this is subject to the constraints that the solving step is orthogonal to the derivatives in $x$ and $z$:
$$
\begin{cases}
\left(\Delta u, \dfrac{\partial u }{\partial x}\right) &= 0\\
\left(\Delta u, \dfrac{\partial u }{\partial z}\right) &= 0
\end{cases}
$$

### Galerkin Projection
We now substitute the basis expansion $u = \xi_j \Psi_j$ into this new equation:

$$\left( c_x \dfrac{\partial \Psi_j}{\partial x} + c_z \dfrac{\partial \Psi_j}{\partial z} \right) \xi_j = -\nabla p - y \, \xi_j \dfrac{\partial \Psi_j }{\partial x} - \xi_j \Psi_{j,v} \, e_x + \dfrac{1}{Re} \xi_j \nabla^2 \Psi_j - \xi_j \xi_k (\Psi_j \cdot \nabla \Psi_k)$$

Taking the inner product of the entire equation with a test function $\Psi_i$ gives:

$$\left(\Psi_i, c_x \dfrac{\partial \Psi_j}{\partial x} + c_z \dfrac{\partial \Psi_j}{\partial z}\right) \xi_j = -(\Psi_i, \nabla p) + \left[ \left(\Psi_i, -y \dfrac{\partial \Psi_j}{\partial x}\right) + \left(\Psi_i, -\Psi_{j,v} e_x) + \dfrac{1}{Re}(\Psi_i, \nabla^2 \Psi_j\right) \right] \xi_j - (\Psi_i, \Psi_j \cdot \nabla \Psi_k) \xi_j \xi_k$$

### Final Algebraic System

After the pressure term vanishes with an application of the divergence theorem and we rename the coefficients $\xi_j$ to $x_j$, the original ODE system becomes a system of algebraic equations. Let's define two new matrices for the terms on the left-hand side:

$$
\begin{align}
C_{x,ij} &= (\Psi_i, \dfrac{\partial \Psi_j}{\partial x}) \\
C_{z,ij} &= (\Psi_i, \dfrac{\partial \Psi_j}{\partial z})
\end{align}
$$

additionally, let 

$$
\begin{align}
A_{1,ij} &= (\Psi_i, -y \, \partial \Psi_j/\partial x) \\
A_{2,ij} &= (\Psi_i, - \Psi_{j,v} \, e_x) \\
A_{3,ij} &= (\Psi_i, \nabla^2 \Psi_j)\\
N_{jkl} &= (\Psi_i, -\Psi_j  \cdot \nabla \Psi_k)
\end{align}
$$

The final system is thus:

$$\left[ c_x C_{x,ij} + c_z C_{z,ij} \right] x_j = \left[ A_{1,ij} + A_{2,ij} + \dfrac{1}{Re} A_{3,ij} \right] x_j + N_{ijk} x_j x_k$$

### Implementing the Phase Constraints

Now, we substitute the Galerkin expansion, $u = \sum\limits_i x_i \psi_i$, into the first constraint equation. We are substituting the expansion for both $u$ and $\partial u/\partial x$:

$$\left( \sum_{i} \Delta x_i \Psi_i, \dfrac{\partial}{\partial x} \left( \sum_{j} x_j \Psi_j \right) \right) = 0$$

Since the coefficients $\Delta x_i$ and $x_j$ are scalars, and differentiation is a linear operator, we can pull them out of the inner product integral:

$$\sum_{i} \sum_{j} \Delta x_i x_j \left(\Psi_i, \dfrac{\partial \Psi_j}{\partial x}\right) = 0$$

The term inside the summation: $(\psi_i, \partial\psi_j / \partial x)$ is the definition of the `(i,j)`-th element of $C_x$.

$$C_{x, ij} = \left(\Psi_i, \dfrac{\partial \Psi_j}{\partial x} \right)$$

So, we can rewrite the entire equation using this matrix notation:
$$\sum_{i} \sum_{j} x_i C_{x, ij} x_j = 0$$

It can be written concisely in matrix-vector notation as:

$$\Delta \vec{x}^T \mathbf{C_x} \vec{x} = 0$$

The exact same procedure can be done for the phase constraint along $z$, resulting in the constraint
$$
\Delta \vec{x}^T \mathbf{C_z} \vec{x} = 0
$$

## Specification of basis set

The basis set $\{ \Psi_i(x,y,z) : i=1,2,\ldots n\}$ should
  * individually satisfy zero divergence: $\nabla \cdot \Psi_i = 0$,
  * individually satisfy the no-slip boundary conditions at the walls: $\nabla \cdot \Psi_i = 0$ and $\Psi_i(x,\pm 1,z) = 0$,
  * individually satisfy the periodic boundary conditions in $x$ and $z$,
  * be either symmetric or antisymmetric with respect to reflection symmetry in $x$, $y$, and $z$ e.g. $\Psi_i(-x,y,z) = \Psi_i(x,y,z)$ or $\Psi_i(-x,y,z)$
  * be complete; i.e. be able to represent all allowable velocity fields in the limit $n \rightarrow \infty$.

Fourier modes in $x,z$ and polynomials in $y$ are the natural choice. The trick is to meet the 
divergence, no-slip, and completeness conditions simultaneously. The basis elements represent 
the velocity field $u = [u,v,w]$ and so have three components, $\Psi_i = [\Psi_{i,u}, \Psi_{i,v}, \Psi_{i,w}]$, or 

\begin{align*}
\Psi_i(x,y,z) = 
\begin{bmatrix}
\Psi_{i,u}(x,y,z) \\
\Psi_{i,v}(x,y,z) \\
\Psi_{i,w}(x,y,z)
\end{bmatrix}
\end{align*}



The basis functions $\Psi_{ijkl}(x,y,z)$ are defined as 

\begin{align*}
\Psi_{1jkl}(x,y,z) &=
\begin{bmatrix} 
E_k(z)\, S_l'(y) \\ 
0\\ 
0 
\end{bmatrix} \quad  \quad \quad \quad \quad \; \;
\text{for} \quad
\begin{matrix}
j = 0,\\
-K \leq k \leq K, \\
0 \leq l \leq L
\end{matrix} \\
\\
\Psi_{2jkl}(x,y,z) &= 
\begin{bmatrix}
0  \\
\gamma k \; E_{k}(z)\,  S_{l}(y) \\
\quad \; \; E_{-k}(z)\, S_{l}'(y)
\end{bmatrix}  \quad \quad \quad
\text{for} \quad 
\begin{matrix}
j=0,\\
1 \leq |k| \leq K,\\
1 \leq l \leq L
\end{matrix} \\
\\
\Psi_{3jkl}(x,y,z) &=
\begin{bmatrix} 
0\\ 
0\\ 
E_j(x)\, S_l'(y)
\end{bmatrix} \quad \quad \quad \quad \quad \; \;
\text{for} \quad
\begin{matrix}
-J \leq j \leq J,\\
k=0, \\
0 \leq l \leq L
\end{matrix} \\
\\
\Psi_{4jkl}(x,y,z) &= 
\begin{bmatrix}
\quad \; \; E_{-j}(x)\, S_{l}'(y) \\
\alpha j \; E_{j}(x)\,  S_{l}(y) \\
0 
\end{bmatrix}  \quad \quad \quad
\text{for} \quad 
\begin{matrix}
1 \leq |j| \leq J,\\
k=0,\\
1 \leq l \leq L
\end{matrix} \\
\\
\Psi_{5jkl}(x,y,z) &=
\begin{bmatrix} 
\; \gamma k \; E_{-j}(x) E_k(z)\, S_l'(y) \\ 
0\\ 
\!-\alpha j \; E_j(x) E_{-k}(z)\, S_l'(y) 
\end{bmatrix} \quad \quad
\text{for} \quad 
\begin{matrix}
1 \leq |j| \leq J,\\
1 \leq |k| \leq K, \\
0 \leq l \leq L
\end{matrix} \\
\\
\Psi_{6jkl}(x,y,z) &= 
\begin{bmatrix}
\quad \gamma k \; E_{-j}(x)\, E_k(z)\, S_{l}'(y) \\
\!\!\!\!\!\! 2 \alpha \gamma j k \; E_{j}(x)\, E_k(z) \; S_{l}(y) \\
\quad \alpha j \; E_{j}(x)\, E_{-k}(z) S_{l}'(y)\
\end{bmatrix} \quad
\text{for} \quad
\begin{matrix}
1 \leq |j| \leq J,\\
1 \leq |k| \leq K,\\
1 \leq l \leq L
\end{matrix}
\end{align*}

where

\begin{align*}
E_j(x) &= \begin{cases} 
            \cos(\alpha j x), & j<0 \\ 
            1, & j = 0 \\
            \sin(\alpha j x), & j>0
          \end{cases}, \quad \quad \quad 
E_k(z) &= \begin{cases} 
            \cos(\gamma k z ), & k<0 \\ 
            1, & j = 0 \\
            \sin(\gamma k z), & k>0
         \end{cases}
\end{align*}


and

\begin{align*}
S_l(y) &= \begin{cases} 
          y-y^3/3, & \quad l=0 \\ 
          (1-y^2)^2 P_{l-1}(y), & \quad l\geq 1
          \end{cases} \\
\end{align*}

where $P_k(y)$ is the $k$th Legendre polynomial. This construction makes $\{S_l(y) : l \geq 1\}$ a basis for $v(y)$, 
and $\{S_l'(y) : l \geq 0\}$ a basis for $u(y)$ and $w(y)$, where $u(y), v(y), w(y)$ are the wall-normal variations of 
each $x,z$ Fourier mode. This follows from 

  * $S_l(y)$ is an $(l+3)$th-order polynomial for $l\geq0$.
  * $S_l(y)$ satisfies $v$ wall BCs $v(\pm 1) = v'(\pm 1)$ for $l\geq 1$. 
  * $S_1(y) = (1-y^2)^2$ is 4th-order, and that's the lowest nonzero polynomial that satisfies the $v$ BCs.
  * $S_l'(y)$ is an $(l+2)$th-order polynomial for $l\geq0$.
  * $S_l'(y)$ satisfies the $u,w$ BCs $u(\pm 1) = w(\pm 1) = 0$ for $l\geq 0$.
  * $S_0'(y) = 1-y^2$ is 2nd-order, and that's the lowest-order nonzero polynomials that satisfies $u,w$ BCs.
  
Thus $\{S_l(y) : l \geq 1\}$ is a basis for $v(y)$, 
and $\{S_l'(y) : l \geq 0\}$ is a basis for $u(y)$ and $w(y)$.


To build a set of basis functions we limit the $x,y,z$ discretization levels respectively
with some fixed $J,K,L$ and 

\begin{align*}
-J \leq j \leq J, \quad 0 \leq k \leq K, \quad -L \leq l \leq L
\end{align*}

For a given $J,K,L$ the number of modes is 
\begin{align*}
N =  (2J+1)(2K+1)(2L+1) + 1
\end{align*}

### Explanation for Vanishing Pressure Term

The pressure term $(\Psi_i, \nabla p)$ vanishes because of the specific properties chosen for the basis functions $\Psi_i$ and the boundary conditions of the domain.

The term is given by the integral:
$$(\Psi_i, \nabla p) = \dfrac{1}{|\Omega|} \int_{\Omega} \Psi_i \cdot \nabla p \; d\Omega$$

**Step 1: Use a vector calculus identity.**
We use the product rule for divergence: $\nabla \cdot (p \Psi_i) = (\nabla p) \cdot \Psi_i + p (\nabla \cdot \Psi_i)$. Rearranging this gives:
$$\Psi_i \cdot \nabla p = \nabla \cdot (p \Psi_i) - p (\nabla \cdot \Psi_i)$$

**Step 2: Substitute the identity into the integral.**
The integral splits into two parts:
$$\int_{\Omega} \Psi_i \cdot \nabla p \; d\Omega = \int_{\Omega} \nabla \cdot (p \Psi_i) \; d\Omega - \int_{\Omega} p (\nabla \cdot \Psi_i) \; d\Omega$$

**Step 3: Analyze the first integral.**
We can apply the **Divergence Theorem** to the first term, which converts the volume integral into a surface integral over the boundary of the domain $\partial\Omega$:
$$\int_{\Omega} \nabla \cdot (p \Psi_i) \; d\Omega = \oint_{\partial\Omega} (p \Psi_i) \cdot \mathbf{n} \; dS$$
where $\mathbf{n}$ is the outward-pointing normal vector on the boundary.
* **On the walls ($y=\pm1$):** The basis functions are required to satisfy the no-slip condition, so $\Psi_i = 0$ on the walls. This makes the integral over the walls zero.
* **On the periodic boundaries ($x$ and $z$):** The flow variables (like $p$ and $\Psi_i$) have the same value on opposite faces of the periodic domain (e.g., at $x=0$ and $x=L_x$). However, the outward normal vectors point in opposite directions ($\mathbf{n} = -e_x$ at $x=0$ and $\mathbf{n} = +e_x$ at $x=L_x$). Because the function values are identical but the normals are opposite, the integrals over these opposing faces exactly cancel each other out.
Therefore, the entire surface integral is zero.

**Step 4: Analyze the second integral.**
The second term is $-\int_{\Omega} p (\nabla \cdot \Psi_i) \; d\Omega$.
A crucial requirement for the basis functions $\Psi_i$ is that they are **divergence-free**, i.e., $\nabla \cdot \Psi_i = 0$. This condition is imposed so that any linear combination of these functions, such as our velocity fluctuation $u = \sum \xi_j \Psi_j$, will also be divergence-free and automatically satisfy the incompressibility constraint $\nabla \cdot u = 0$.
Since $\nabla \cdot \Psi_i = 0$ everywhere in the domain $\Omega$, the integrand $p (\nabla \cdot \Psi_i)$ is zero, and thus the second integral is also zero.

Since both integrals are zero, the entire pressure term $(\Psi_i, \nabla p)$ vanishes. This elegant result allows us to eliminate the pressure from the final system of equations, which is a major simplification.